# All Data - NLP EDA - NER on Company Names

(Slow file separated out from other code)

In [ ]:
# Flag for if you are running this on the sample dataset
# Sample comprises 100 earnings calls (transcripts included)
# Full data comprises 4532 earnings calls (transcripts included)
sample = False
# Modify this path as needed to run on your machine
sample_path = r'~\Box\STAT 222 Capstone\Intermediate Data\all_data_sample.csv'

In [ ]:
# Packages
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns

In [ ]:
# Load in sample csv, or full parquet file
# Use inputted sample path, or ~\Box\STAT 222 Capstone\Intermediate Data\all_data.parquet
if sample:
    df = pd.read_csv(sample_path)
else:
    df = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\all_data.parquet') 
df

## NER with Spacy - number of companies


In [ ]:
import spacy
from tqdm.auto import tqdm # progress tracking
tqdm.pandas()

# load model and disable unnecessary components
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'lemmatizer', 'attribute_ruler', 'morphologizer', 'textcat'])

# Function to process transcript and count company mentions
def count_companies(text):
    # Process the text
    doc=nlp(text)
    # Counter
    count = 0
    # Initialize count
    for entity in doc.ents:
        # We'll select just entities that are tagged organizations (close enough to companies)
        if entity.label_ == "ORG":
            count += 1
    return count

# Apply the function to the transcript column
# potentially look into ways to do this in parallel or make more efficient...
df['company_mentions'] = df['transcript'].progress_apply(count_companies)

# Plot the distribution of company mentions
plt.figure(figsize=(8, 5))
plt.hist(df['company_mentions'], bins=20, color='skyblue', edgecolor='black')
plt.title('Company Mentions Distribution')
plt.xlabel('Number of Company Mentions')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

print('Average number of company mentions:', df['company_mentions'].mean())

In [ ]:
# Testing the nlp model
sample_text = 'Microsoft is a great company. I also like Apple and Google.'
print('testing on sample text:')
print(sample_text)
doc = nlp(sample_text)
for entity in doc.ents:
    print(entity.text, entity.label_)
harder_sample_text = 'I am going to the bank to deposit my money. apple had a great quarter.'
print('testing on harder sample text:')
print(harder_sample_text)
doc = nlp(harder_sample_text)
for entity in doc.ents:
    print(entity.text, entity.label_)